In [227]:
import os
import sys

def adding_module_path():
    module_path = os.path.abspath(os.path.join('..'))

    if module_path not in sys.path:
        sys.path.append(module_path)

adding_module_path()

In [228]:
from src.data_preparing.build.gutenberg_builder import GutenbergBuilder
from src.app.project_setup import ProjectSetup
from src.data_preparing.split.run_split_deps_on_stats import run_split_deps_on_stats_same_dir
from src.utils.create_path_to_gutenberg import create_path_to_gutenberg_sentence_authors_sentence, create_path_to_gutenberg_authors
from src.utils.create_test_dataset_from import create_test_dataset_from
from src.data_preparing.split.run_split_deps_on_stats import run_split_deps_on_stats_same_dir
from src.utils.create_path_to_gutenberg import create_path_to_gutenberg_sentence_authors_sentence
from src.config.config import PATH_TO_DATASET_FOLDER_TEST, PATH_TO_DATASET_FOLDER_TEST, AUTHORS_FILE_NAME

from src.tokenizers.prepare_dataset_from_tokenizer import prepare_dataset_from_tokenizer
from src.tokenizers.transformer_tokenizer import TransformerTokenizer
import tensorflow as tf
from src.encoder.create_encoder_from_path import create_encoder_from_path
from src.testing.get_testing_dataset import dataset as test_dataset
from src.data_loading.get_dataset_object_from import get_dataset_object_from_path, get_datasets
from src.utils.create_path_to_gutenberg import get_paths_to_gutenberg, get_path_to_gutenberg_sets

In [229]:
PATH_TO_DATASET_FOLDER_TEST

'/home/usp/pro0255/diploma/data_test'

In [230]:
from transformers import TFAutoModel
from src.experiments.settings import LearningSettings
from transformers import AutoConfig
from src.types.transformer_name import TransformerName
from src.types.processing_type import PreprocessingType
from src.preprocessing.preprocessing_factory import PreprocessingFactory
from src.types.transformer_pooling import TransformerPooling
from src.config.run_prep import run_prep
run_prep()

Running preperation of packages


[nltk_data] Downloading package punkt to
[nltk_data]     /home/usp/pro0255/diploma/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/usp/pro0255/diploma/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/usp/pro0255/diploma/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [231]:
settings = LearningSettings()

In [232]:
model_name = TransformerName.BertBaseUncased.value

In [233]:
preprocessing_factory = PreprocessingFactory()

Creating shorting method with min = 3
Creating lemma method with instance <WordNetLemmatizer>


In [234]:
config = AutoConfig.from_pretrained(model_name, output_hidden_states=True)

In [214]:
path_data, path_authors = get_path_to_gutenberg_sets(10, 3, PATH_TO_DATASET_FOLDER_TEST)

In [215]:
path_data

('/home/usp/pro0255/diploma/data_test/gutenberg/10Authors/Sentence3/train.csv',
 '/home/usp/pro0255/diploma/data_test/gutenberg/10Authors/Sentence3/valid.csv',
 '/home/usp/pro0255/diploma/data_test/gutenberg/10Authors/Sentence3/test.csv')

In [235]:
train, valid, test = get_datasets(path_data, ';', preprocessing_factory.create(PreprocessingType.Default))

In [236]:
for x in test:
    print(x)
    break

(<tf.Tensor: shape=(), dtype=string, numpy=b'looked laughed recognized strange guest approached unobserved new comer deformed little man big year old boy big head oldish uncommonly sharply cut feature noblest egyptian kept house dwarf sport little wight served wife mena capacity'>, <tf.Tensor: shape=(), dtype=int32, numpy=1800>)


In [295]:
tokenizer = TransformerTokenizer(
    model_name, 
    create_encoder_from_path(
        path_authors
    )
)

In [160]:
class TransformerPooling(Enum):
    LastHiddenState = "last_hidden_state"
    Pooler = "pooler_output"
    HiddenStates = "hidden_states"
    Blank = BLANK_DESCRIPTION
    #TODO create own others

NameError: name 'Enum' is not defined

In [378]:
transformer = TFAutoModel.from_config(config)

In [379]:
input_ids = tf.keras.layers.Input(shape=(512, ), name='input_ids', dtype='int32')
mask = tf.keras.layers.Input(shape=(512, ), name='attention_mask', dtype='int32')

embeddings = transformer(input_ids, attention_mask=mask)  # we only keep tensor 0 (last_hidden_state)
X = BertPoolingLayer()(embeddings, TransformerPooling.HiddenStates)
#X = BertEmbeddingLayer()(embeddings)
#X = tf.keras.layers.GlobalMaxPool1D()(embeddings)  # reduce tensor dimensionality
#X = tf.keras.layers.BatchNormalization()(X)

X = tf.keras.layers.Dense(128, activation='relu')(X)
X = tf.keras.layers.Dropout(0.1)(X)
y = tf.keras.layers.Dense(10, activation='softmax', name='outputs')(X)  # adjust based on number of sentiment classes

model = tf.keras.Model(inputs=[input_ids, mask], outputs=y)

A


ValueError: Exception encountered when calling layer "bert_pooling_layer_130" (type BertPoolingLayer).

in user code:

    File "/tmp/ipykernel_40288/3776596310.py", line 26, in call  *
        cls = layers_together[:, 0, :]

    ValueError: Index out of range using input dim 2; input has only 2 dims for '{{node bert_pooling_layer_130/cond/strided_slice}} = StridedSlice[Index=DT_INT32, T=DT_FLOAT, begin_mask=5, ellipsis_mask=0, end_mask=5, new_axis_mask=0, shrink_axis_mask=2](bert_pooling_layer_130/cond/Mean, bert_pooling_layer_130/cond/strided_slice/stack, bert_pooling_layer_130/cond/strided_slice/stack_1, bert_pooling_layer_130/cond/strided_slice/stack_2)' with input shapes: [512,768], [3], [3], [3] and with computed input tensors: input[3] = <1 1 1>.


Call arguments received:
  • inputs=TFBaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state='tf.Tensor(shape=(None, 512, 768), dtype=float32)', pooler_output='tf.Tensor(shape=(None, 768), dtype=float32)', past_key_values=None

In [380]:
model.layers[2].trainable = True

In [381]:
model.summary()

Model: "model_29"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 512)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 512)]        0           []                               
                                                                                                  
 tf_bert_model_23 (TFBertModel)  TFBaseModelOutputWi  109482240  ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 512,                                        

In [382]:
model.compile(loss=settings.loss, optimizer=settings.optimizer, metrics=settings.metric)

In [94]:
history = model.fit(
   prepare_dataset_from_tokenizer(train, tokenizer)
    .shuffle(buffer_size=30000)
    .cache()
    .batch(32)
    .prefetch(4),
    #validation_data=prepare_dataset_from_tokenizer(valid, tokenizer).batch(1),
    epochs=2
)

Epoch 1/2
      2/Unknown - 67s 26s/step - loss: 2.8906 - accuracy: 0.1042

KeyboardInterrupt: 

In [275]:
res = model.predict(prepare_dataset_from_tokenizer(test, tokenizer).take(10).batch(1))

In [276]:
res.shape

(10, 10)

In [277]:
import numpy as np

In [278]:
np.argmax(res, axis=1)

array([0, 0, 0, 8, 0, 0, 0, 0, 8, 0])

In [376]:
class BertPoolingLayer(tf.keras.layers.Layer):
 
    def call(self, inputs, pooling_type):
        if pooling_type == TransformerPooling.LastHiddenState:
            last_hidden_state = inputs[TransformerPooling.LastHiddenState.value]
            print(last_hidden_state)
            return tf.reduce_mean(last_hidden_state, axis=1)
            
            
        if pooling_type == TransformerPooling.Pooler:
            pooler = inputs[TransformerPooling.Pooler.value]
            return pooler
        
        
        if pooling_type == TransformerPooling.HiddenStates:
            #TODO: deal with hidden state
            selector = inputs[TransformerPooling.HiddenStates.value]
            
            layers = tf.convert_to_tensor(selector)[-1]
            
            if tf.shape(tf.shape(layers)) > 3:
                #print(layers)
                print('A')
                layers_together = tf.reduce_mean(layers, axis=0)
                average_sentence_words = tf.reduce_mean(layers_together, axis=1)
                cls = layers_together[:, 0, :]
                return cls
                
                
                #print(average_sentence_words)
                #layers = tf.reduce_mean(layers, axis=0, keepdims=False)
                #cls = layers[:, 0, :]
                #average_sentence_words = tf.reduce_mean(layers[:, 1:tf.shape(layers)[1]-1, :], axis=1)
                #return average_sentence_words
            else:
                print('B')
                
                #return tf.reduce_mean(layers, axis=1)
                cls = layers[:, 0, :]
                #return cls
                average_sentence_words = tf.reduce_mean(layers[:, 1:tf.shape(layers)[1]-1, :], axis=1)
                return average_sentence_words

In [377]:
for x in prepare_dataset_from_tokenizer(train, tokenizer).batch(10):
    text, label = x
    output = transformer(text, output_hidden_states=True)
    output = BertPoolingLayer()(output, TransformerPooling.HiddenStates)
    #print(output)
    break

B


In [372]:
for x in prepare_dataset_from_tokenizer(dataset, tokenizer).batch(10):
    print(x)
    break

NameError: name 'dataset' is not defined

In [45]:
a

{'input_ids': array([ 101, 7592, 1045, 2572,  102,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,

Run model experiment with to lower

... find best preprocessing

# Test ideas

- one dataset - combination of preprocessing
- size of input (k) - deps on bert - make statistics
- type of transformer - distilbert, larbe bert etc
- head of transformer - 
- encoding of transformer
- classic model of tranformer
- testování proti Glove LSTM, Embedding Dense, Glove LSTM, Glove Dense
